<a href="https://colab.research.google.com/github/seihoon0111/rcs/blob/main/23_3%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CF 알고리즘 (협업 필터링)


In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

base_src = "drive/MyDrive/recosys/data"
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id','age','sex','occupation','zip_code']

users = pd.read_csv(u_user_src,
                    sep = '|',
                    names = u_cols,
                    encoding = 'latin_1')
users = users.set_index('user_id')

u_item_src = os.path.join(base_src,'u.item')

i_cols = ['movie_id','title','release date','video release data',
          'IMDB URL','unknown','Action','Adventure','Animation',
          'Children\'s','Comedy','Crime','Documentary','Drama','Fantasy',
          'Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']

movies = pd.read_csv(u_item_src,
                     sep ='|',
                     names = i_cols,
                     encoding = 'latin-1')
movies = movies.set_index('movie_id')

u_data_src = os.path.join(base_src,'u.data')

r_cols = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(u_data_src,
                      sep ='\t',
                      names = r_cols,
                      encoding='latin-1')

x= ratings.copy()
y= ratings['user_id']

x_train,x_test,y_train,y_test = train_test_split(x,y,
                                                 test_size=0.25,
                                                 stratify = y)

ratings_matrix = x_train.pivot(index='user_id',columns='movie_id',values='rating')

def RMSE(y_true,y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

def score(model,neighbor = 0):
  id_pair = zip(x_test['user_id'],x_test['movie_id'])
  y_pred = np.array([model(user,movie,neighbor) for (user,movie) in id_pair])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true,y_pred)

cos similarity CF simple

In [ ]:
ratings_matrix = x_train.pivot(index='user_id',columns='movie_id',values='rating')
from sklearn.metrics.pairwise import cosine_similarity

matrix_dummy = ratings_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy,matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = ratings_matrix.index, columns = ratings_matrix.index)

def CF_simple(user_id,movie_id,neighbor):
  if movie_id in ratings_matrix.columns:
    sim_score = user_similarity[user_id].copy()
    movie_ratings = ratings_matrix[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    sim_score = sim_score.drop(none_rating_idx)
    if np.sum(sim_score) != 0:
      cf_score = np.dot(sim_score, movie_ratings)/np.sum(sim_score)
    else:
      cf_score = 3.0
  else:
    cf_score = 3.0

  return cf_score

score(CF_simple)

1.0178571307169724

CF neighbor (KNN)

In [ ]:
ratings_matrix = x_train.pivot(index='user_id',columns='movie_id',values='rating')
matrix_dummy = ratings_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy,matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = ratings_matrix.index, columns = ratings_matrix.index)

def CF_KNN(user_id, movie_id, neighbor):
  if neighbor==0:
    return CF_simple(user_id,movie_id,neighbor)
  else:
    if movie_id in ratings_matrix.columns:
      sim_scores = user_similarity[user_id].copy()
      movie_ratings = ratings_matrix[movie_id].copy()
      none_rating_idx = movie_ratings[movie_ratings.isnull()].index
      movie_ratings = movie_ratings.dropna()
      sim_scores = sim_scores.drop(none_rating_idx)
      if len(sim_scores)>1:
        neighbor_size = min(neighbor,len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        mean_rating = np.dot(sim_scores,movie_ratings)/np.sum(sim_scores)
      else:
        mean_rating = 3.0
    else:
      mean_rating = 3.0
    return mean_rating
for neighbor in [10,20,30,40,50,60]:
  m = score(CF_KNN,neighbor)
  n = neighbor
  print("%.4f %d"%(m,n))


score(CF_KNN,neighbor = 30)

1.0244 10
1.0114 20
1.0092 30
1.0088 40
1.0100 50
1.0106 60


1.0092371183654256

real model recommendation

In [ ]:
ratings_matrix = x.pivot(index='user_id',columns='movie_id',values='rating')
matrix_dummy = ratings_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy,matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = ratings_matrix.index, columns = ratings_matrix.index)


def recom_movie(user_id,n_items,neighbor = 30):
  user_movie = ratings_matrix.loc[user_id].copy()
  for movie_id in ratings_matrix.columns:
    if pd.notnull(user_movie.loc[movie_id]):
      user_movie.loc[movie_id] = 0
    else:
      user_movie.loc[movie_id] = CF_KNN(user_id,movie_id,neighbor)
  movie_sort = user_movie.sort_values(ascending = False)[:n_items]
  recom_movies = movies.loc[movie_sort.index]
  recommendation = recom_movies['title']
  return recommendation


recom_movie(7,5,30)


movie_id
1467    Saint of Fort Washington, The (1993)
1189                      Prefontaine (1997)
1293                         Star Kid (1997)
1500               Santa with Muscles (1996)
1463                        Boys, Les (1997)
Name: title, dtype: object